<a href="https://colab.research.google.com/github/Tinynja/Sarsa-phi-EB/blob/main/rl_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install dependencies
!pip install matplotlib numpy gym

# Run this cell

# type hinting 
from typing import Sequence, Tuple, Dict, Any, Optional

import numpy as np


# env
import gym
from gym.wrappers import Monitor

# data manipulation, colab dispaly, and plotting
import pandas as pd
import matplotlib.pyplot as plt


# misc util
import random, glob, base64, itertools
from pathlib import Path
from pprint import pprint

In [6]:
#######################################################################
# Copyright (C)                                                       #
# 2017-2018 Shangtong Zhang(zhangshangtong.cpp@gmail.com)             #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from math import floor
from tqdm import tqdm


# all possible actions
ACTIONS = range(4)

# discount is always 1.0 in these experiments
DISCOUNT = 1.0

# use optimistic initial value, so it's ok to set epsilon to 0
EPSILON = 0.05

# maximum steps per episode
STEP_LIMIT = 5000


# get action at @position and @velocity based on epsilon greedy policy and @valueFunction  #########################    use our own get_action. modified it, may work as intended
def get_action(observation, valueFunction):
    if np.random.binomial(1, EPSILON) == 1:
        return np.random.choice(ACTIONS)
    values = []
    for action in ACTIONS:
        values.append(valueFunction.value(observation))  
    return np.argmax(values)



# replacing trace update rule
# @trace: old trace (will be modified)
# @activeTiles: current active tile indices
# @lam: lambda
# @return: new trace for convenience
def replacing_trace(trace, activeTiles, lam):
    active = np.in1d(np.arange(len(trace)), activeTiles)
    trace[active] = 1
    trace[~active] *= lam * DISCOUNT
    return trace



# wrapper class for Sarsa(lambda)
class Sarsa:
    # In this example I use the tiling software instead of implementing standard tiling by myself
    # One important thing is that tiling is only a map from (state, action) to a series of indices
    # It doesn't matter whether the indices have meaning, only if this map satisfy some property
    # View the following webpage for more information
    # http://incompleteideas.net/sutton/tiles/tiles3.html
    # @maxSize: the maximum # of indices
    #the hashing is a lfa?
    def __init__(self, step_size, lam, trace_update=replacing_trace, max_size=28672):
        self.max_size = max_size
        self.trace_update = trace_update
        self.lam = lam

        # divide step size equally to each tiling
        self.step_size = step_size/10

        # weight for each tile
        self.weights = np.zeros(max_size) #max size is the number of features?

        # trace for each tile
        self.trace = np.zeros(max_size)



    # estimate the value of given state and action
    def value(self, observation):
        active_tiles = np.nonzero(observation)
        return np.sum(self.weights[active_tiles])

    # learn with given state, action and target
    def learn(self, observation, target):
        active_tiles = np.nonzero(observation)
        estimation = np.sum(self.weights[active_tiles])
        delta = target - estimation
        print('target: ' + str(target))
        #print('estimation array: ' + str(self.weights[active_tiles]))
        print('estimation: ' + str(np.sum(self.weights[active_tiles])))
        if self.trace_update == replacing_trace:
            self.trace_update(self.trace, active_tiles, self.lam)
        else:
            raise Exception('Unexpected Trace Type')
        self.weights += self.step_size * delta * self.trace
        print('delta: ' + str(delta))
        print('traces: ' + str(self.trace))
        print('weights: ' +  str(self.weights))


# play Mountain Car for one episode based on given method @evaluator
# @return: total steps in this episode
def play(evaluator, env):

    action = random.choice(ACTIONS)
    steps = 0
    while True:
        next_observation, reward, done, info = env.step(action)
        next_action = get_action(next_observation, evaluator)    #########################    use our own get_action  ??? modified it, may work as intented
        steps += 1
        target = reward + DISCOUNT * evaluator.value(next_observation)          ############# use our own value function ??? modified it, may work as intented
        evaluator.learn(observation, target)
        observation = next_observation
        action = next_action
        if done:
            break
        if steps >= STEP_LIMIT:
            print('Step Limit Exceeded!')
            break
    return steps

In [7]:
class BaseAgent:
  """ The base agent class function.
  """
  def __init__(self, nb_features=28672):
    #nothing for now
    self.gamma = 1
    self.features = nb_features
    self.rhos = np.ones(self.features) #stores the rho_i values


  def takeAction(self, t):
    phis = [[0,1,0],[0,1,0],[0,1,0],[1,0,1]]
    return phis[t]


  def updateRho_i(self, counts, t):
    M = self.features
    for i in range(M): #M is the number of features of phi
      counts[i] += 1 #since we add phi to the seen states, all the counts are increased by one for t+1
      self.rhos[i] = (counts[i]+0.5)/(t+1)
    return 0


  def PHI_EB(self, evaluator, env, beta=0.05, t_end=200):
    t = 0
    M = self.features #number of features
    counts = np.zeros(M)
    states = np.zeros((t_end,M)) #stores the previous phis for all timesteps

    action = 1 #starting the game for the agent on the first game
    old_phi = env._observe()
    print('starting iterations')
    while t < t_end:
      #observe phi(s), reward
      phi, reward, done, info = env.step(action)
      next_action = get_action(phi, evaluator)
      print('phi: ' + str(phi))
      
      #compute rho_t(phi) (feature visit-density)
      if t > 0:
        rho_t = 1
        for i in range(M):
          counts[i] = 0
          for step in range(t):
            if phi[i] == states[step,i]:
              counts[i] += 1
          self.rhos[i] = (counts[i]+0.5)/(t+1)
          rho_t = rho_t*self.rhos[i]
      else:
        rho_t = 0.5**M
      print('rho_t ' + str(rho_t))
      #update all rho_i with observed phi
      states[t] = phi
      self.updateRho_i(counts, t+1)
      print('min rho_i_t: ' + str(min(self.rhos)))
      
      #compute rho_t+1(phi)
      new_rho_t = 1
      for i in range(M):
        new_rho_t = new_rho_t*self.rhos[i]
      if new_rho_t <= 1e-323: #this is to avoid division by zero, might need to be tweaked
        new_rho_t = 1e-323
      print('new_rho_t ' + str(new_rho_t))

      #compute Nhat_t(s)
      Nhat_t = rho_t*(1-new_rho_t)/(new_rho_t-rho_t)
      if Nhat_t <= 1e-323: #this is to avoid division by zero again, might need to be tweaked
        Nhat_t = 1e-323

      #compute R(s,a) (empirical reward)
      explorationBonus = beta/np.sqrt(Nhat_t)

      print(explorationBonus)
      reward = reward + explorationBonus

      print('weights: ' + str(evaluator.weights))
      print(max(evaluator.weights))
      print('state value: ' + str(evaluator.value(phi)))
      #pass phi(s) and reward to RL algo to update theta_t
      target = reward + self.gamma * evaluator.value(phi)          ############# use our own value function ??? modified it, may work as intented
      evaluator.learn(old_phi, target)

      if done:
        #break
        env.reset()
        action = 1
        old_phi = env.observe()
      else:
        old_phi = phi
        action = next_action

      t += 1

    return evaluator.weights


In [8]:
from ale_py.roms import Breakout
env = EnvALE(Breakout, feature_type='Basic')
print(env.action_space)
alpha = 0.25
lam = 0.99
evaluator = Sarsa(alpha, lam, replacing_trace,28672)
agent = BaseAgent()
done, observation = False, env.reset(do_record=True)
weights = agent.PHI_EB(evaluator, env, beta=0.05, t_end=100)
env.show_video()

ModuleNotFoundError: ignored